In [340]:
import json
from gridappsd.simulation import Simulation # Import Simulation Library
from gridappsd import GridAPPSD
import time
from gridappsd.topics import simulation_input_topic
from gridappsd import topics as t


In [341]:
username = "system"
password = "manager"

# Note: there are other parameters for connecting to
# systems other than localhost
gapps = GridAPPSD(username=username, password=password)


checking for connection to gridappsd

In [342]:
assert gapps.connected

adding the run_config for simulation setup

In [204]:
run123_config = json.load(open("Run123NodeFileSimAPI.json")) # Pull simulation start message from saved file

In [205]:
simulation_obj = Simulation(gapps, run123_config) # Create Simulation object
simulation_obj.start_simulation() # Start Simulation

simulation_id = simulation_obj.simulation_id # Obtain Simulation ID
print("Successfully started simulation with simulation_id: ", simulation_id)

Successfully started simulation with simulation_id:  1835457202


taking simulation id from the sim itself, or externally provided

In [343]:
# viz_simulation_id = simulation_id
viz_simulation_id = 841527766
topic = simulation_input_topic(viz_simulation_id)


start, stop or resume

In [ ]:
ch = int(input())
if ch == 1:
    message = {"command": "pause"}
    gapps.send(topic, message)
elif ch == 2:
    message = {"command": "resume"}
    gapps.send(topic, message)
elif ch == 3:
    message = {"command": "stop"}
    gapps.send(topic, message)

else:
    exit()

Taking Measurements 

In [ ]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectType": "LinearShuntCompensator"
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message) # Pass query to PowerGrid Models API
measurements_obj = response_obj["data"]
print(measurements_obj)
global Pos_obj # Define global python dictionary of position measurements
Pos_obj = [k for k in measurements_obj if k['type'] == 'Pos'] # Filter measurements to just switch positions

print(Pos_obj) # Print switch position measurement mRID for first switch

In [344]:
# Use queryFilter of "startTime" and "endTime"
message = {
    "queryMeasurement":"weather",
    "queryFilter":{"startTime":"1357048800000000",
                   "endTime":"1357048860000000"},
    "responseFormat":"JSON"
}

response_obj = gapps.get_response(t.TIMESERIES, message) # Pass query to Timeseries API
weather_obj = response_obj["data"]

print(weather_obj[1]) # Print first line of weather data

{'Diffuse': 5.1879076, 'AvgWindSpeed': 6.6437, 'TowerRH': 36.9, 'long': '105.18 W', 'MST': '14:01', 'TowerDryBulbTemp': 30.0326, 'DATE': '1/1/2013', 'DirectCH1': 86.69558059999999, 'GlobalCM22': 37.990990499999995, 'AvgWindDirection': 115.4, 'time': 1357048860, 'place': 'Solar Radiation Research Laboratory', 'lat': '39.74 N'}


subscribing to output

In [354]:
def demoSubscription1(header, message):
    # Extract time and measurement values from message
    timestamp = message["message"]["timestamp"]
    meas_value = message["message"]["measurements"]

    meas_mrid = list(meas_value.keys()) #obtain list of all mrid from message

    # Filter to measurements with value of zero
    open_switches = []
    for index in Pos_obj:
        if index["measid"] in meas_value:
            mrid = index["measid"]
            power = meas_value[mrid]
            if power["value"] == 0:
                open_switches.append(index["eqname"])

    # Print message to command line
    print("............")
    print("Number of open switches at time", timestamp, ' is ', len(set(open_switches)))

In [355]:
from gridappsd.topics import simulation_output_topic

output_topic = simulation_output_topic(viz_simulation_id)

conn_id = gapps.subscribe(output_topic, demoSubscription1)

............
Number of open switches at time 1640370132  is  0
............
Number of open switches at time 1640370132  is  0
............
Number of open switches at time 1640370135  is  0
............
Number of open switches at time 1640370135  is  0
............
Number of open switches at time 1640370138  is  0
............
Number of open switches at time 1640370138  is  0
............
Number of open switches at time 1640370141  is  0
............
Number of open switches at time 1640370141  is  0
............
Number of open switches at time 1640370144  is  0
............
Number of open switches at time 1640370144  is  0
............
Number of open switches at time 1640370147  is  0
............
Number of open switches at time 1640370147  is  0
............
Number of open switches at time 1640370150  is  0
............
Number of open switches at time 1640370150  is  0
............
Number of open switches at time 1640370153  is  0
............
Number of open switches at time 1640370153

unsubscribing from simulation

In [351]:
gapps.unsubscribe(conn_id)

storing the mrid

In [330]:
model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
switch_dict = response_obj["data"]
# print(switch_dict)
sw_mrid=''

for index in switch_dict:
    if index["IdentifiedObject.name"] == 'sw1':
        sw_mrid = index["IdentifiedObject.mRID"]
print(sw_mrid)

_7CBC54BB-4A93-410F-AF92-DDA633676AA0


publishing inputs to the sim using DifferenceBuilder

In [ ]:
import time
from gridappsd import DifferenceBuilder
input_topic = simulation_input_topic(viz_simulation_id)

model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" # model mrid of IEEE123 sim

my_open_diff = DifferenceBuilder(viz_simulation_id)
my_open_diff.add_difference(sw_mrid, "Switch.open", 1, 0) # Open switch given by sw_mrid
open_message = my_open_diff.get_message()
gapps.send(input_topic, open_message)

# time.sleep(10)

# my_close_diff = DifferenceBuilder(viz_simulation_id)
# my_close_diff.add_difference(sw_mrid, "Switch.open", 0, 1) # Close switch given by sw_mrid
# close_message = my_close_diff.get_message()
# gapps.send(input_topic, close_message)


input using mesages

In [ ]:
input_topic = simulation_input_topic(viz_simulation_id)
message = {
  "command": "update",
  "input": {
      "simulation_id": str(viz_simulation_id),
      "message": {
          "timestamp": int(time.time()),
          "difference_mrid": "_abcd1234",
          "reverse_differences": [{

                  "object": sw_mrid,
                  "attribute": "Switch.open",
                  "value": 1
              }
          ],
          "forward_differences": [{

                  "object": sw_mrid,
                  "attribute": "Switch.open",
                  "value": 0
              }
              ]
              }
      }
}
gapps.send(input_topic, message)
time.sleep(1)
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}
response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
switch_dict = response_obj["data"]
for index in switch_dict:
    if index["IdentifiedObject.name"] == 'sw1':
        print(index)



inputs for capacitors

In [293]:

message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "objectType": "LinearShuntCompensator"
}
while True:
    time.sleep(2)
    response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA, message)
#     print(response_obj)
    cap_dict = response_obj["data"]
    for index in cap_dict:
        if index['IdentifiedObject.name'] == 'c92c':
            print(index, index["ShuntCompensator.sections"])



{'id': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'ConductingEquipment.BaseVoltage': '_C26F5029-D0AF-4301-B8D5-7A9767B95DEE', 'Equipment.EquipmentContainer': '_C1C3E687-6FFD-C753-582B-632A27E28507', 'IdentifiedObject.mRID': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'IdentifiedObject.name': 'c92c', 'PowerSystemResource.Location': '_A57616AD-93BB-4B6D-8A7E-743CD74BCE4D', 'LinearShuntCompensator.b0PerSection': '0', 'LinearShuntCompensator.bPerSection': '0.008666106', 'LinearShuntCompensator.g0PerSection': '0', 'LinearShuntCompensator.gPerSection': '0', 'ShuntCompensator.aVRDelay': '0', 'ShuntCompensator.grounded': 'true', 'ShuntCompensator.maximumSections': '1', 'ShuntCompensator.nomU': '2402', 'ShuntCompensator.normalSections': '1', 'ShuntCompensator.phaseConnection': 'PhaseShuntConnectionKind.Y', 'type': 'LinearShuntCompensator'}
{'id': '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', 'ConductingEquipment.BaseVoltage': '_C26F5029-D0AF-4301-B8D5-7A9767B95DEE', 'Equipment.EquipmentContainer': '_C1C3

KeyboardInterrupt: 